In [2]:
#modules import
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz

from sklearn.metrics import accuracy_score, auc, confusion_matrix, f1_score, precision_score, recall_score, roc_curve

import prettytable

import pydotplus
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
#import labelled data to train classification model
labelled_data = pd.read_csv('labelled_whitelist_shop2.csv')
labelled_data.shape

labelled_data.info()
# print(labelled_data['kmeans.labels'].value_counts())
# print(labelled_data['agglomerative.average.labels'].value_counts())
# print(labelled_data['agglomerative.complete.labels'].value_counts())
# print(labelled_data['meanshift.labels'].value_counts())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536 entries, 0 to 535
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   manage_shop_indicator          536 non-null    float64
 1   officialstore_indicator        536 non-null    float64
 2   preferred_shop_indicator       536 non-null    float64
 3   crossborder_indicator          536 non-null    float64
 4   shop_category                  536 non-null    float64
 5   new_seller_flag                536 non-null    float64
 6   seller_centre_login_L30D       536 non-null    float64
 7   shop_sku_number                536 non-null    float64
 8   shop_follower_number           536 non-null    float64
 9   shop_L180D_order               536 non-null    float64
 10  weighted_shop_rating           536 non-null    float64
 11  kmeans.labels                  536 non-null    int64  
 12  agglomerative.average.labels   536 non-null    int

In [4]:
#import unlabelled data
non_whitelist = pd.read_csv('non_whitelist_filled.csv')
nw_data = non_whitelist.groupby("shop_index").mean()
nw_data = nw_data.drop(columns=['decorated_indicator','Unnamed: 0','performance_date','masked_item_impression','masked_order','masked_shop_page_view','masked_shop_click_from_search','masked_campaign_tab_click','masked_other_tab_click'])
nw_data.head(20)
nw_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26018 entries, 119700 to 1486953315
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   manage_shop_indicator     26018 non-null  float64
 1   officialstore_indicator   26018 non-null  float64
 2   preferred_shop_indicator  26018 non-null  float64
 3   crossborder_indicator     26018 non-null  float64
 4   shop_category             26018 non-null  float64
 5   new_seller_flag           26018 non-null  float64
 6   seller_centre_login_L30D  26018 non-null  float64
 7   shop_sku_number           26018 non-null  float64
 8   shop_follower_number      26018 non-null  float64
 9   shop_L180D_order          26018 non-null  float64
 10  weighted_shop_rating      26018 non-null  float64
dtypes: float64(11)
memory usage: 2.4 MB


In [5]:
#define x,y -> agglomerative.average.labels 
x = labelled_data.iloc[:,:11]
y = labelled_data.iloc[:,-5]
#get train/test data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=2018)

In [6]:
#decision tree classifier
dt_model = DecisionTreeClassifier(random_state=2018)
dt_model.fit(x_train, y_train)

#test the model with test dataset
pre_y = dt_model.predict(x_test)
accuracy_s = accuracy_score(y_test, pre_y)
print("DTC classifier: ",accuracy_s)
y_score = dt_model.predict_proba(x_test)
y_score

#prediction on non-whitelist data
prediction_DTC = dt_model.predict(nw_data)
prediction_DTC
prob_DTC = dt_model.predict_proba(nw_data)
prob_DTC[0:20]

DTC classifier:  0.8757763975155279


array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.]])

In [20]:
#random forest classifier 
from sklearn.ensemble import RandomForestClassifier
# Create the model with 100 trees
rf_model = RandomForestClassifier(n_estimators=100, bootstrap = True, max_features = 'sqrt')

# Fit on training data
rf_model.fit(x_train, y_train)

# Actual class predictions
rf_predictions = rf_model.predict(x_test)

# Probabilities for each class
rf_probs =rf_model.predict_proba(x_test)[:, 1]
# rf_probs
rf_predictions

# test accuracy 
test2=accuracy_score(y_test,rf_predictions)
# print("RF accuracy based on test data",test2)

#prediction on non_whitelist data
prediction_RF = rf_model.predict(nw_data)
print(prediction_RF[20:40])
proba_RF = rf_model.predict_proba(nw_data)
# print("RF probability for non-whitelist data:",proba_RF)
proba_RF = list(proba_RF)
count = 0
for i in proba_RF:
    if max(i)<0.7:
        count += 1
# print("unmatched num(RF): ",count)

# add in classification label
import copy
X = copy.deepcopy(nw_data)
rf_labels = rf_model.predict(X)
# print(rf_labels)
rf_probs = rf_model.predict_proba(X)
print(rf_probs)
rf_prob = [np.max(i) for i in rf_probs]
X.loc[:,'rf_label'] = rf_labels
X.loc[:,'rf_probs'] = rf_prob


len(rf_probs)
X.head()




[0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
[[0.47 0.46 0.   0.06 0.01]
 [0.89 0.01 0.06 0.   0.04]
 [0.33 0.64 0.   0.03 0.  ]
 ...
 [0.3  0.02 0.63 0.01 0.04]
 [0.45 0.04 0.47 0.01 0.03]
 [0.96 0.   0.02 0.   0.02]]


,manage_shop_indicator,officialstore_indicator,preferred_shop_indicator,crossborder_indicator,shop_category,new_seller_flag,seller_centre_login_L30D,shop_sku_number,shop_follower_number,shop_L180D_order,weighted_shop_rating,rf_label,rf_probs
shop_index,,,,,,,,,,,,,
119700,1.0,0.0,0.0,0.0,15.0,0.0,26.0,9311.0,58590.0,25079.0,4.0,0,0.47
207225,0.0,0.0,0.0,0.0,1.0,0.0,2.0,269.0,1029.0,470.0,2.0,0,0.89
215775,0.0,0.0,0.0,0.0,6.0,0.0,28.0,3052.0,15684.0,13197.0,3.0,1,0.64
308340,0.0,0.0,0.0,0.0,9.0,0.0,2.0,423.0,1143.0,509.0,4.0,0,0.97
350280,0.0,0.0,0.0,0.0,1.0,0.0,11.0,467.0,1641.0,522.0,5.0,0,0.93


In [28]:
#GradientBoosting Classifier
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_curve,auc
from sklearn import metrics
from sklearn.model_selection import GridSearchCV 
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
ss_x = StandardScaler()
ss_y = StandardScaler()
train_x = ss_x.fit_transform(x_train)
test_x = ss_x.transform(x_test)

#model training
model_GBDT = GradientBoostingClassifier(random_state=10)
model_GBDT.fit(train_x,y_train)
y_pred = model_GBDT.predict(train_x)
print("")
y_predprob = model_GBDT.predict_proba(train_x)[:,1]


#predict on non whitelist data
model_GBDT.predict(nw_data)
prob_GBDT = model_GBDT.predict_proba(nw_data)
prob_GBDT
count_GBDT = 0
for i in range(len(prob_GBDT)):
    if np.max(prob_GBDT[i])<0.8:
        count_GBDT += 1
# print(count_GBDT)

#GB labels assignment; 
X2 = copy.deepcopy(nw_data)

GB_label = model_GBDT.predict(X.iloc[:,:11])
print(GB_label)
GB_probs = model_GBDT.predict_proba(X.iloc[:,:11])
GB_prob = [np.max(i) for i in GB_probs]
print(GB_prob)
X.loc[:,'rf_label'] = rf_labels
X.loc[:,'rf_probs'] = rf_prob

X.loc[:,'GBDT_label'] = rf_labels
X.loc[:,'GBDT_probs'] = rf_prob

X.head(20)


[0 0 0 ... 0 0 0]
[0.8119116327923376, 0.80791934985067, 0.7910867573249691, 0.7910867573249691, 0.80791934985067, 0.7910867573249691, 0.7910867573249691, 0.9101013615354286, 0.8013022647801775, 0.7910867573249691, 0.7910867573249691, 0.8013022647801775, 0.7910867573249691, 0.7910867573249691, 0.7910867573249691, 0.7910867573249691, 0.7910867573249691, 0.7910867573249691, 0.7910867573249691, 0.7910867573249691, 0.7910867573249691, 0.7910867573249691, 0.7910867573249691, 0.7910867573249691, 0.7910867573249691, 0.7910867573249691, 0.7910867573249691, 0.8013022647801775, 0.7910867573249691, 0.80791934985067, 0.7910867573249691, 0.9101013615354286, 0.7910867573249691, 0.7910867573249691, 0.9101013615354286, 0.7910867573249691, 0.7910867573249691, 0.7910867573249691, 0.7910867573249691, 0.7910867573249691, 0.7910867573249691, 0.9101013615354286, 0.8119116327923376, 0.7910867573249691, 0.7910867573249691, 0.7910867573249691, 0.7910867573249691, 0.7910867573249691, 0.8013022647801775, 0.7910

,manage_shop_indicator,officialstore_indicator,preferred_shop_indicator,crossborder_indicator,shop_category,new_seller_flag,seller_centre_login_L30D,shop_sku_number,shop_follower_number,shop_L180D_order,weighted_shop_rating,rf_label,rf_probs,GBDT_label,GBDT_probs
shop_index,,,,,,,,,,,,,,,
119700,1.0,0.0,0.0,0.0,15.0,0.0,26.0,9311.0,58590.0,25079.0,4.000000,0,0.47,0,0.47
207225,0.0,0.0,0.0,0.0,1.0,0.0,2.0,269.0,1029.0,470.0,2.000000,0,0.89,0,0.89
215775,0.0,0.0,0.0,0.0,6.0,0.0,28.0,3052.0,15684.0,13197.0,3.000000,1,0.64,1,0.64
308340,0.0,0.0,0.0,0.0,9.0,0.0,2.0,423.0,1143.0,509.0,4.000000,0,0.97,0,0.97
350280,0.0,0.0,0.0,0.0,1.0,0.0,11.0,467.0,1641.0,522.0,5.000000,0,0.93,0,0.93
396360,0.0,0.0,0.0,0.0,5.0,0.0,1.0,412.0,5700.0,457.0,2.000000,0,0.93,0,0.93
406305,0.0,0.0,0.0,0.0,8.0,0.0,5.0,566.0,2139.0,1757.0,2.000000,0,0.62,0,0.62
412875,0.0,0.0,0.0,0.0,20.0,0.0,1.0,247.0,1143.0,405.0,0.000000,0,0.94,0,0.94
505260,0.0,0.0,0.0,0.0,0.0,0.0,2.0,841.0,2205.0,1848.0,4.000000,0,0.51,0,0.51


In [29]:
#export classification results
X.to_csv(r'nw_classification_results.csv',index = False)